In [1]:
import pandas as pd
import numpy as np

In [2]:
def convert_to_year(x):
    val = int(x)
    if val >= 19:
        return 1900 + val
    else:
        return 2000 + val

In [3]:
data_path = '../data/'
songs = pd.read_csv(data_path + 'songs.csv')
songs_extra = pd.read_csv(data_path + 'song_extra_info.csv')
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
song_extra_cols = ['song_id', 'song_name', 'isrc']
songs_extra['song_country'] = songs_extra['isrc'].apply(lambda x: -1 if x == -1 or x == '' or pd.isnull(x) else str(x)[:2])
songs_extra['song_year'] = songs_extra['isrc'].apply(lambda x: -1 if x == -1 or x == '' or pd.isnull(x)\
                                                         else int(convert_to_year(x[5:7])))
songs1 = songs.merge(songs_extra[songs_extra.columns], on='song_id', how='left')

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
members = pd.read_csv(data_path + 'members.csv')

train = train.merge(songs1[songs1.columns], on='song_id', how='left')
test = test.merge(songs1[songs1.columns], on='song_id', how='left')

## artist_year : 
### given a artist, return the earlist year of their songs.

## non_artists:
### key is the artists whose all songs are incorrect.
### given a artist, return how many user listen to their songs

## user_artists:
### given a user, return the artists this user listen to

## user_song_years
### given a user, return the average year of songs this user listen to

In [5]:
artist_year = {}
#given a artist, return the earlist year of their songs.
for song, artist, year in zip(songs1['song_name'], songs1['artist_name'], songs1['song_year']):
    if year >= 1960:
        artist_year[artist] = min(year, artist_year.get(artist, 3000))

In [6]:
non_artists = {}
#key is the artists whose all songs are incorrect.
#given a artist, return how many user listen to their songs
user_artists = {}
#given a user, return the artists this user listen to
for uid, artist, year in zip(train['msno'], songs1['artist_name'], songs1['song_year']):
    if uid not in user_artists:
        user_artists[uid] = set([])
    user_artists[uid].add(artist)
    if year < 1960:
        predict_year = artist_year.get(artist, 3000)
        if predict_year == 3000:
            if artist not in non_artists:
                non_artists[artist] = {}
            non_artists[artist][uid] = non_artists[artist].get(uid, 0) + 1

In [7]:
user_song_years = {}
#given a user, return the average year of songs this user listen to
for uid in user_artists:
    res = []
    for artist in user_artists[uid]:
        if artist in artist_year and\
                    artist_year[artist] != 3000:
            res +=  artist_year[artist],
    user_song_years[uid] = np.mean(res) if len(res) > 0 else 0

In [8]:
for artist in non_artists:
    res = []
    for uid in non_artists[artist]:
        if user_song_years[uid] > 0:
            res += user_song_years[uid],
    artist_year[artist] = np.mean(res) if len(res) > 0 else 0

In [9]:
songs_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']

In [10]:
songs_info = songs_extra.merge(songs[songs.columns], on='song_id', how='left')
print 'ok'

ok


In [11]:
for x in list(songs1):
    print x

song_id
song_length
genre_ids
artist_name
composer
lyricist
language
song_name
isrc
song_country
song_year


In [12]:
years = []
for artist, year in zip(songs1['artist_name'], songs1['song_year']):
    try:
        year = int(year)
    except:
        if artist in artist_year:
            years += int(artist_year[artist]),
        else:
            years += -1,
        continue
    if year >= 1960 and year <= 2018:
        years += year,
    else:     
        years += int(artist_year[artist]),

In [13]:
dict = {}
for year in years:
    dict[year] = dict.get(year, 0) + 1
for x in dict:
    print x, dict[x]

1960 12683
1961 2999
1962 2561
1963 2147
1964 3059
1965 2109
1966 2058
1967 2543
1968 2702
1969 2479
1970 2276
1971 2878
1972 3248
1973 2548
1974 2859
1975 3534
1976 3110
1977 3305
1978 3718
1979 3712
1980 3789
1981 4434
1982 4542
1983 5199
1984 5127
1985 5392
1986 5805
1987 6791
1988 15564
1989 14095
1990 15762
1991 15126
1992 16066
1993 12566
1994 14247
1995 16783
1996 18789
1997 21583
1998 58331
1999 97907
2000 48421
2001 43309
2002 37333
2003 47730
2004 59615
2005 58123
2006 65601
2007 68766
2008 83111
2009 91534
2010 114978
2011 107109
2012 132166
2013 138168
2014 214817
2015 271397
2016 255967
2017 29972
2018 3721
-1 56


In [70]:
print len(songs1['song_year']), len(years)

2296320 2296320


In [72]:
songs1['song_year'] = years

In [73]:
songs1.to_csv(data_path + 'songs1.csv', sep=',', encoding='utf-8')

In [77]:
songs1 =  pd.read_csv(data_path + 'songs1.csv')

In [78]:
for x in list(songs1):
    print x

id
song_id
song_length
genre_ids
artist_name
composer
lyricist
language
song_name
isrc
song_country
song_year


In [17]:
print len(non_song), len(non_artists)

97252 6573
